## Introduction to Rcognita for dynamical systems simulation

* Config
* Pipeline
    * System
    * Predictor
    * Actor
    * Critic
    * Models
    * Optimizers
    * Controller
    * Simulator
    * Scenario
    

## Config

pipelines.config_blueprints.py

In [ ]:
import numpy as np
import argparse
from abc import abstractmethod, ABC
from rcognita_framework.rcognita.utilities import rc
from rcognita_framework.rcognita import (
    systems, 
    predictors, 
    actors, 
    critics, 
    models, 
    optimizers, 
    controllers, 
    simulator, 
    scenarios,
    animators
)
import torch
from copy import deepcopy
%matplotlib notebook

### Default argument parser

In [ ]:
class LoadFromFile(argparse.Action):
    def __call__(self, parser, namespace, values, option_string=None):
        with values as f:
            # parse arguments in the file and store them in the target namespace
            parser.parse_args(f.read().split(), namespace)

class RcognitaArgParser(argparse.ArgumentParser):
    def __init__(self, description):

        super().__init__(description=description)
        self.add_argument(
            "--control_mode",
            metavar="control_mode",
            type=str,
            choices=["manual", "nominal", "MPC", "RQL", "SQL", "STAG"],
            default="MPC",
            help="Control mode. Currently available: "
            + "----manual: manual constant control specified by action_manual; "
            + "----nominal: nominal controller, usually used to benchmark optimal controllers;"
            + "----MPC:model-predictive control; "
            + "----RQL: Q-learning actor-critic with prediction_horizon-1 roll-outs of stage objective; "
            + "----SQL: stacked Q-learning; "
            + "----STAG: joint actor-critic (stabilizing), system-specific, needs proper setup.",
        )
        self.add_argument(
            "--is_log",
            action="store_true",
            help="Flag to log data into a data file. Data are stored in simdata folder.",
        )
        self.add_argument(
            "--no_visual",
            action="store_true",
            help="Flag to produce graphical output.",
        )
        self.add_argument(
            "--no_print",
            action="store_true",
            help="Flag to print simulation data into terminal.",
        )
        self.add_argument(
            "--is_est_model",
            action="store_true",
            help="Flag to estimate environment model.",
        )
        self.add_argument(
            "--save_trajectory",
            action="store_true",
            help="Flag to store trajectory inside the pipeline during execution.",
        )
        self.add_argument(
            "--dt",
            type=float,
            metavar="sampling_time",
            dest="sampling_time",
            default=0.01,
            help="Controller sampling time.",
        )
        self.add_argument(
            "--t1",
            type=float,
            metavar="time_final",
            dest="time_final",
            default=10.0,
            help="Final time of episode.",
        )
        self.add_argument("--config", type=open, action=LoadFromFile)
        self.add_argument(
            "strings", metavar="STRING", nargs="*", help="String for searching",
        )

        self.add_argument(
            "-f",
            "--file",
            help="Path for input file. First line should contain number of lines to search in",
        )

### Config metaclass

What it basically does is collecting of arguments parsed from sys.argv and save into local namespace afterwards

In [ ]:
class MetaConf(type):
    def __init__(cls, name, bases, clsdict):
        if "argument_parser" in clsdict:

            def new_argument_parser(self):
                args = clsdict["argument_parser"](self)
                self.__dict__.update(vars(args))

            setattr(cls, "argument_parser", new_argument_parser)

In [ ]:
class AbstractConfig(object, metaclass=MetaConf):
    @abstractmethod
    def __init__(self):
        self.config_name = None

    @abstractmethod
    def argument_parser(self):
        pass

    @abstractmethod
    def pre_processing(self):
        pass

    def get_env(self):
        self.argument_parser()
        self.pre_processing()
        return self.__dict__
    
    def config_to_pickle(self):
        with open(
            f"../tests/refs/env_{self.config_name}.pickle", "wb"
        ) as env_description_out:
            pickle.dump(self.__dict__, env_description_out) ### Dump environment

### Config for 3-wheel-robot

In [ ]:
class Config3WRobotNI(AbstractConfig):
    def __init__(self):
        self.config_name = "3wrobot_NI"

    def argument_parser(self):
        description = "Agent-environment pipeline: a 3-wheel robot (kinematic model a. k. a. non-holonomic integrator)."

        parser = RcognitaArgParser(description=description)

        parser.add_argument(
            "--Nruns",
            type=int,
            default=1,
            help="Number of episodes. Learned parameters are not reset after an episode.",
        )
        parser.add_argument(
            "--state_init",
            type=str,
            nargs="+",
            metavar="state_init",
            default=["5", "5", "-3*pi/4"],
            help="Initial state (as sequence of numbers); "
            + "dimension is environment-specific!",
        )
        parser.add_argument(
            "--model_est_stage",
            type=float,
            default=1.0,
            help="Seconds to learn model until benchmarking controller kicks in.",
        )
        parser.add_argument(
            "--model_est_period_multiplier",
            type=float,
            default=1,
            help="Model is updated every model_est_period_multiplier times sampling_time seconds.",
        )
        parser.add_argument(
            "--model_order",
            type=int,
            default=5,
            help="Order of state-space estimation model.",
        )
        parser.add_argument(
            "--prob_noise_pow",
            type=float,
            default=False,
            help="Power of probing (exploration) noise.",
        )
        parser.add_argument(
            "--action_manual",
            type=float,
            default=[-5, -3],
            nargs="+",
            help="Manual control action to be fed constant, system-specific!",
        )
        parser.add_argument(
            "--prediction_horizon",
            type=int,
            default=3,
            help="Horizon length (in steps) for predictive controllers.",
        )
        parser.add_argument(
            "--pred_step_size_multiplier",
            type=float,
            default=1.0,
            help="Size of each prediction step in seconds is a pred_step_size_multiplier multiple of controller sampling time sampling_time.",
        )
        parser.add_argument(
            "--data_buffer_size",
            type=int,
            default=10,
            help="Size of the buffer (experience replay) for model estimation, agent learning etc.",
        )
        parser.add_argument(
            "--running_obj_struct",
            type=str,
            default="quadratic",
            choices=["quadratic", "biquadratic"],
            help="Structure of stage objective function.",
        )
        parser.add_argument(
            "--R1_diag",
            type=float,
            nargs="+",
            default=[1, 10, 1, 0, 0],
            help="Parameter of stage objective function. Must have proper dimension. "
            + "Say, if chi = [observation, action], then a quadratic stage objective reads chi.T diag(R1) chi, where diag() is transformation of a vector to a diagonal matrix.",
        )
        parser.add_argument(
            "--R2_diag",
            type=float,
            nargs="+",
            default=[1, 10, 1, 0, 0],
            help="Parameter of stage objective function . Must have proper dimension. "
            + "Say, if chi = [observation, action], then a bi-quadratic stage objective reads chi**2.T diag(R2) chi**2 + chi.T diag(R1) chi, "
            + "where diag() is transformation of a vector to a diagonal matrix.",
        )
        parser.add_argument(
            "--discount_factor", type=float, default=1.0, help="Discount factor."
        )
        parser.add_argument(
            "--critic_period_multiplier",
            type=float,
            default=1.0,
            help="Critic is updated every critic_period_multiplier times sampling_time seconds.",
        )
        parser.add_argument(
            "--critic_struct",
            type=str,
            default="quad-nomix",
            choices=["quad-lin", "quadratic", "quad-nomix", "quad-mix", "NN"],
            help="Feature structure (critic). Currently available: "
            + "----quad-lin: quadratic-linear; "
            + "----quadratic: quadratic; "
            + "----quad-nomix: quadratic, no mixed terms; "
            + "----quad-mix: quadratic, mixed observation-action terms (for, say, action_objective or advantage function approximations)."
            + "----NN: Torch neural network.",
        )
        parser.add_argument(
            "--actor_struct",
            type=str,
            default="container",
            choices=["quad-lin", "quadratic", "quad-nomix", "container"],
            help="Feature structure (actor). Currently available: "
            + "----quad-lin: quadratic-linear; "
            + "----quadratic: quadratic; "
            + "----quad-nomix: quadratic, no mixed terms."
            + "----container: container, just to store actions",
        )

        args = parser.parse_args()
        return args

    def pre_processing(self):
        self.trajectory = []
        self.dim_state = 3
        self.dim_input = 2
        self.dim_output = self.dim_state
        self.dim_disturb = 2
        if self.control_mode == "STAG":
            self.prediction_horizon = 1

        self.dim_R1 = self.dim_output + self.dim_input
        self.dim_R2 = self.dim_R1
        # ----------------------------------------Post-processing of arguments
        # Convert `pi` to a number pi
        for k in range(len(self.state_init)):
            self.state_init[k] = eval(self.state_init[k].replace("pi", str(np.pi)))

        self.state_init = np.array(self.state_init)
        self.action_manual = np.array(self.action_manual)

        self.pred_step_size = self.sampling_time * self.pred_step_size_multiplier
        self.model_est_period = self.sampling_time * self.model_est_period_multiplier
        self.critic_period = self.sampling_time * self.critic_period_multiplier

        self.R1 = np.diag(np.array(self.R1_diag))
        self.R2 = np.diag(np.array(self.R2_diag))

        assert self.time_final > self.sampling_time > 0.0
        assert self.state_init.size == self.dim_state

        # ----------------------------------------(So far) fixed settings
        self.is_disturb = 0
        self.is_dynamic_controller = 0

        self.time_start = 0

        self.action_init = 0 * np.ones(self.dim_input)

        # Solver
        self.atol = 1e-5
        self.rtol = 1e-3

        # xy-plane
        self.xMin = -10
        self.xMax = 10
        self.yMin = -10
        self.yMax = 10

        # Model estimator stores models in a stack and recall the best of model_est_checks
        self.model_est_checks = 0

        # Control constraints
        self.v_min = -25
        self.v_max = 25
        self.omega_min = -5
        self.omega_max = 5
        self.action_bounds = np.array(
            [[self.v_min, self.v_max], [self.omega_min, self.omega_max]]
        )

        self.xCoord0 = self.state_init[0]
        self.yCoord0 = self.state_init[1]
        self.angle0 = self.state_init[2]
        self.angle_deg_0 = self.angle0 / 2 / np.pi
        self.observation_target = []

### Pipeline: System

In [ ]:
class Sys3WRobotNISeminar(systems.System):
    """
    System class: 3-wheel robot with static actuators (the NI - non-holonomic integrator).


    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.name = "3wrobotNI"

        if self.is_disturb:
            self.sigma_disturb = self.pars_disturb[0]
            self.mu_disturb = self.pars_disturb[1]
            self.tau_disturb = self.pars_disturb[2]

    def _compute_state_dynamics(self, time, state, action, disturb=[]):

        Dstate = rc.zeros(self.dim_state)

        if self.is_disturb and (disturb != []):
            Dstate[0] = action[0] * rc.cos(state[2]) + disturb[0]
            Dstate[1] = action[0] * rc.sin(state[2]) + disturb[0]
            Dstate[2] = action[1] + disturb[1]
        else:
            Dstate[0] = action[0] * rc.cos(state[2])
            Dstate[1] = action[0] * rc.sin(state[2])
            Dstate[2] = action[1]

        return Dstate

    def out(self, state, time=None, action=None):

        return state
    
    
    def compute_closed_loop_rhs(self, time, state_full):
        rhs_full_state = rc.zeros(self._dim_full_state, prototype=state_full)

        state = state_full[0 : self.dim_state]

        if self.is_disturb:
            disturb = state_full[self.dim_state :]
        else:
            disturb = []

        if self.is_dynamic_controller:
            action = state_full[-self.dim_input :]
            observation = self.out(state)
            rhs_full_state[-self.dim_input :] = self._ctrlDyn(time, action, observation)
        else:
            # Fetch the control action stored in the system
            action = self.action

        rhs_full_state[0 : self.dim_state] = self._compute_state_dynamics(
            time, state, action, disturb
        ).T

        if self.is_disturb:
            rhs_full_state[self.dim_state :] = self._compute_disturbance_dynamics(
                time, disturb
            ).T

        # Track system's state
        self._state = state

        return rhs_full_state

### Pipeline: Predictor

In [ ]:
class EulerPredictorSeminar(predictors.EulerPredictor):
    """
    Euler predictor uses a simple Euler discretization scheme.
    It does predictions by increments 
    scaled by a sampling time times the velocity evaluated at each successive node.

    """

    def predict(self, current_state_or_observation, action):
        next_state_or_observation = (
            current_state_or_observation
            + self.pred_step_size
            * self.compute_state_dynamics([], current_state_or_observation, action)
        )
        return next_state_or_observation

    def predict_sequence(self, observation, action_sequence):

        observation_sequence = rc.zeros(
            [self.prediction_horizon, self.dim_output], prototype=action_sequence
        )
        current_observation = observation

        for k in range(self.prediction_horizon):
            current_action = action_sequence[k, :]
            next_observation = self.predict(current_observation, current_action)
            observation_sequence[k, :] = self.sys_out(next_observation)
            current_observation = next_observation
        return observation_sequence

### Pipeline: Optimizers

In [ ]:
class SciPyOptimizer(optimizers.BaseOptimizer):
    engine = "SciPy"

    def __init__(self, opt_method, opt_options, verbose=True):
        self.opt_method = opt_method
        self.opt_options = opt_options
        self.verbose = verbose

    @optimizers.BaseOptimizer.verbose
    def optimize(self, objective, initial_guess, bounds, constraints=(), verbose=True):

        weight_bounds = sp.optimize.Bounds(bounds[0], bounds[1], keep_feasible=True)

        before_opt = objective(initial_guess)
        opt_result = minimize(
            objective,
            x0=initial_guess,
            method=self.opt_method,
            bounds=weight_bounds,
            options=self.opt_options,
            constraints=constraints,
            tol=1e-7,
        )
        if verbose:
            print(f"before:{before_opt},\nafter:{opt_result.fun}")

        return opt_result.x


class CasADiOptimizer(optimizers.BaseOptimizer):
    engine = "CasADi"

    def __init__(self, opt_method, opt_options, verbose=True):
        self.opt_method = opt_method
        self.opt_options = opt_options
        self.verbose = verbose

    @optimizers.BaseOptimizer.verbose
    def optimize(
        self,
        objective,
        initial_guess,
        bounds,
        constraints=(),
        decision_variable_symbolic=None,
    ):
        optimization_problem = {
            "f": objective,
            "x": rc.concatenate(decision_variable_symbolic),
            "g": (),
        }

        if isinstance(constraints, tuple):
            upper_bound_constraint = [0 for _ in constraints]
        elif isinstance(constraints, (SX, DM, int, float)):
            upper_bound_constraint = [0]

        try:
            solver = nlpsol(
                "solver", self.opt_method, optimization_problem, self.opt_options,
            )
        except Exception as e:
            print(e)
            return initial_guess

        if upper_bound_constraint is not None and len(upper_bound_constraint) > 0:
            result = solver(
                x0=initial_guess,
                lbx=bounds[0],
                ubx=bounds[1],
                ubg=upper_bound_constraint,
            )
        else:
            result = solver(x0=initial_guess, lbx=bounds[0], ubx=bounds[1])

        return result["x"]

### Pipeline: Actor
Main parts of actor are:
* `objective`
* `model`

In [ ]:
class ActorSeminar(actors.Actor):

    def objective(
        self, action_sequence, observation,
    ):
        action_sequence_reshaped = rc.reshape(
            action_sequence, [self.prediction_horizon + 1, self.dim_input]
        )

        observation_sequence = [observation]

        observation_sequence_predicted = self.predictor.predict_sequence(
            observation, action_sequence_reshaped
        )

        observation_cur = rc.reshape(observation, [1, self.dim_output])

        observation_sequence = rc.vstack(
            (observation_cur, observation_sequence_predicted)
        )

        actor_objective = 0
        for k in range(self.prediction_horizon):
            actor_objective += self.discount_factor ** k * self.running_objective(
                observation_sequence[k, :].T, action_sequence_reshaped[k, :].T
            )
        return actor_objective

    def update(self, observation, constraint_functions=(), time=None):
        """
        Method to update the current action or weight tensor.
        The old (previous) action or weight tensor is stored inside model istself.
        The `time` argument is used for debugging purposes.
        """

        action_sequence_old = rc.rep_mat(
            self.model.cache.weights, 1, self.prediction_horizon + 1
        )

        action_sequence_init_reshaped = rc.reshape(
            action_sequence_old, [(self.prediction_horizon + 1) * self.dim_input,],
        )

        constraints = ()

        if self.optimizer.engine == "CasADi":
            action_sequence_init_reshaped = rc.DM(action_sequence_init_reshaped)

            symbolic_dummy = rc.array_symb((1, 1))

            symbolic_var = rc.array_symb(
                tup=rc.shape(action_sequence_init_reshaped), prototype=symbolic_dummy
            )

            actor_objective = lambda action_sequence: self.objective(
                action_sequence, observation
            )

            actor_objective = rc.lambda2symb(actor_objective, symbolic_var)

            if constraint_functions:
                constraints = self.create_constraints(
                    constraint_functions, symbolic_var, observation
                )

            action_sequence_optimized = self.optimizer.optimize(
                actor_objective,
                action_sequence_init_reshaped,
                self.action_bounds,
                constraints=constraints,
                decision_variable_symbolic=symbolic_var,
            )

        elif self.optimizer.engine == "SciPy":
            actor_objective = rc.func_to_lambda_with_params(
                self.objective, observation, var_prototype=action_sequence_init_reshaped
            )

            if constraint_functions:
                constraints = sp.optimize.NonlinearConstraint(
                    partial(
                        self.create_constraints,
                        constraint_functions=constraint_functions,
                        observation=observation,
                    ),
                    -np.inf,
                    0,
                )

            action_sequence_optimized = self.optimizer.optimize(
                actor_objective,
                action_sequence_init_reshaped,
                self.action_bounds,
                constraints=constraints,
            )

        
        action_obtained = action_sequence_optimized[: self.dim_input]
        self.model.update_and_cache_weights(action_obtained)

### Pipeline: Critic

In [ ]:
class CriticTrivialSeminar(critics.Critic):
    """
    This is a dummy to calculate outcome (accumulated running objective).

    """

    def __init__(self, running_objective, sampling_time=0.01):
        self.running_objective = running_objective
        self.sampling_time = sampling_time
        self.outcome = 0

    def __call__(self):
        return self.outcome

    def objective(self, weights):
        pass

    def get_optimized_weights(self, constraint_functions=(), time=None):
        pass

    def update_buffers(self, observation, action):
        self.update_outcome(observation, action)

    def update(self, constraint_functions=(), time=None):
        pass

    def update_outcome(self, observation, action):

        self.outcome += self.running_objective(observation, action) * self.sampling_time

    def reset(self):
        self.outcome = 0

### Pipeline: Models

In [ ]:
class ModelAbstract(ABC):
    """
    Blueprint of a model.
    """

    def __call__(self, *args, weights=None, use_stored_weights=False):

        if use_stored_weights is False:
            if weights is not None:
                return self.forward(*args, weights=weights)
            else:
                return self.forward(*args, weights=self.weights)
        else:
            return self.cache.forward(*args, weights=self.weights)

    @property
    @abstractmethod
    def model_name(self):
        return "model_name"

    @abstractmethod
    def __init__(self):
        pass

    @abstractmethod
    def forward(self):
        pass

    def update_and_cache_weights(self, weights):
        if "cache" not in self.__dict__.keys():
            self.cache = deepcopy(self)

        self.weights = weights

        self.cache.weights = weights

In [ ]:
class ModelWeightContainerSeminar(ModelAbstract):
    """
    A special model which just basically stores weights

    """

    model_name = "action-sequence"

    def __init__(self, weights_init=None):
        self.weights = weights_init
        self.update_and_cache_weights(self.weights)

    def forward(self, *argin):
        return self.weights

In [ ]:
class ModelQuadNoMix(ModelAbstract):
    """
    Quadratic model (no mixed terms).

    """

    model_name = "quad-nomix"

    def __init__(self, input_dim, sinle_weight_min=1.0, sinle_weight_max=1e3):
        self.dim_weights = input_dim
        self.weight_min = sinle_weight_min * np.ones(self.dim_weights)
        self.weight_max = sinle_weight_max * np.ones(self.dim_weights)
        self.weights = self.weight_min
        self.update_and_cache_weights(self.weights)

    def forward(self, *argin, weights):
        if len(argin) > 1:
            vec = rc.concatenate(tuple(argin))
        else:
            vec = argin[0]

        polynom = vec * vec
        result = rc.dot(weights, polynom)

        return result

In [ ]:
class ModelQuadForm(ModelAbstract):
    """
    Quadratic form.

    """

    model_name = "quad_form"

    def __init__(self, weights=None):
        self.weights = weights

    def forward(self, *argin, weights):

        if len(argin) != 2:
            raise ValueError("ModelQuadForm assumes two vector arguments!")

        vec = rc.concatenate(tuple(argin))

        result = vec.T @ weights @ vec

        result = rc.squeeze(result)

        return result

### Pipeline: Controller

In [ ]:
class OptimalController(ABC):
    """
    A blueprint of optimal controllers.
    """

    def __init__(
        self, time_start=0, sampling_time=0.1, observation_target=[],
    ):

        self.controller_clock = time_start
        self.sampling_time = sampling_time

        self.observation_target = observation_target

    def estimate_model(self, observation, time):
        if self.is_est_model or self.mode in ["RQL", "SQL"]:
            self.estimator.estimate_model(observation, time)

    def reset(self, time_start):
        """
        Resets agent for use in multi-episode simulation.
        Only internal clock and current actions are reset.
        All the learned parameters are retained.

        """
        self.controller_clock = time_start
        self.actor.model.weights = self.actor.action_init
        self.actor.model.update_and_cache_weights(self.actor.action_init)

    def compute_action_sampled(self, time, observation, constraints=()):

        time_in_sample = time - self.controller_clock
        timeInCriticPeriod = time - self.critic_clock
        is_critic_update = timeInCriticPeriod >= self.critic_period

        # CRITIC CLOKC NOT UPDATED!!!! FIX !!!!

        if time_in_sample >= self.sampling_time:  # New sample
            # Update controller's internal clock
            self.controller_clock = time

            action = self.compute_action(
                time, observation, is_critic_update=is_critic_update
            )

            return action

        else:
            return self.actor.model.cache.weights

    @abstractmethod
    def compute_action(self):
        pass


class RLControllerSeminar(OptimalController):
    """
    Reinforcement learning controller class.
    Takes instances of `actor` and `critic` to operate.
    Action computation is sampled, i.e., actions are computed at discrete, equi-distant moments in time.
    `critic` in turn is updated every `critic_period` units of time.
    """

    def __init__(
        self, *args, critic_period=0.1, actor=[], critic=[], time_start=0, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.actor = actor
        self.critic = critic

        self.dim_input = self.actor.dim_input
        self.dim_output = self.actor.dim_output

        self.critic_clock = time_start
        self.critic_period = critic_period

    def compute_action(
        self, time, observation, is_critic_update=False,
    ):
        # Critic

        # Update data buffers
        self.critic.update_buffers(observation, self.actor.model.cache.weights)

        if is_critic_update:
            # Update critic's internal clock
            self.critic_clock = time
            self.critic.update(time=time)

        self.actor.update(observation)
        action = self.actor.get_action()

        return action

### Pipeline: Scenario

In [ ]:
class OnlineScenarioSeminar:
    """
    Online scenario: the controller and system interact with each other live via exchange of observations and actions, successively in time steps.

    """

    def __init__(
        self,
        system,
        simulator,
        controller,
        actor,
        critic,
        logger,
        datafiles,
        time_final,
        running_objective,
        no_print=False,
        is_log=False,
        is_playback=False,
        state_init=None,
        action_init=None,
    ):
        self.system = system
        self.simulator = simulator
        self.controller = controller
        self.actor = actor
        self.critic = critic
        self.logger = logger
        self.time_final = time_final
        self.outcome = 0
        self.datafile = datafiles[0]
        self.running_objective = running_objective
        self.trajectory = []
        self.no_print = no_print
        self.is_log = is_log
        self.time_old = 0
        self.is_playback = is_playback
        self.state_init = state_init
        self.action_init = action_init
        if self.is_playback:
            self.table = []

    def run(self):
        while True:
            self.step()

    def step(self):
        sim_status = self.simulator.do_sim_step()
        if sim_status == -1:
            return -1

        (
            self.time,
            _,
            self.observation,
            self.state_full,
        ) = self.simulator.get_sim_step_data()
        self.trajectory.append(rc.concatenate((self.state_full, self.time), axis=None))
        delta_time = self.time - self.time_old
        self.time_old = self.time

        self.action = self.controller.compute_action_sampled(
            self.time, self.observation
        )
        self.system.receive_action(self.action)

        self.running_objective_value = self.running_objective(
            self.observation, self.action
        )
        self.update_outcome(self.observation, self.action, delta_time)

        if not self.no_print:
            self.logger.print_sim_step(
                self.time,
                self.state_full,
                self.action,
                self.running_objective_value,
                self.outcome,
            )
        if self.is_log:
            self.logger.log_data_row(
                self.datafile,
                self.time,
                self.state_full,
                self.action,
                self.running_objective_value,
                self.outcome,
            )

        if self.is_playback:
            self.table.append(
                [
                    self.time,
                    *self.state_full,
                    *self.action,
                    self.running_objective_value,
                    self.outcome,
                ]
            )

    def update_outcome(self, observation, action, delta):

        """
        Sample-to-sample accumulated (summed up or integrated) stage objective. This can be handy to evaluate the performance of the agent.
        If the agent succeeded to stabilize the system, ``outcome`` would converge to a finite value which is the performance mark.
        The smaller, the better (depends on the problem specification of course - you might want to maximize objective instead).

        """

        self.outcome += self.running_objective(observation, action) * delta

# Test pipeline!

In [ ]:
from rcognita_framework.pipelines.pipeline_3wrobot_NI import Pipeline3WRobotNI

class Pipeline3WRobotNISeminar(Pipeline3WRobotNI):
    config = Config3WRobotNI

    def initialize_system(self):
        self.system = Sys3WRobotNISeminar(
            sys_type="diff_eqn",
            dim_state=self.dim_state,
            dim_input=self.dim_input,
            dim_output=self.dim_output,
            dim_disturb=self.dim_disturb,
            pars=[],
            is_dynamic_controller=self.is_dynamic_controller,
            is_disturb=self.is_disturb,
            pars_disturb=rc.array(
                [
                    [200 * self.sampling_time, 200 * self.sampling_time],
                    [0, 0],
                    [0.3, 0.3],
                ]
            ),
        )
        
    def initialize_predictor(self):
        self.predictor = EulerPredictorSeminar(
            self.pred_step_size,
            self.system._compute_state_dynamics,
            self.system.out,
            self.dim_output,
            self.prediction_horizon,
        )
        
    def initialize_models(self):
        self.dim_critic_model_input = self.dim_input + self.dim_output
        self.critic_model = ModelQuadNoMix(self.dim_critic_model_input)
        self.actor_model = models.ModelWeightContainer(weights_init=self.action_init)
        self.model_running_objective = ModelQuadForm(weights=self.R1)

        A = rc.zeros([self.model_order, self.model_order])
        B = rc.zeros([self.model_order, self.dim_input])
        C = rc.zeros([self.dim_output, self.model_order])
        D = rc.zeros([self.dim_output, self.dim_input])
        initial_guessest = rc.zeros(self.model_order)

        self.model_SS = models.ModelSS(A, B, C, D, initial_guessest)
        
    def initialize_optimizers(self):
        opt_options = {
            "maxiter": 500,
            "maxfev": 5000,
            "disp": False,
            "adaptive": True,
            "xatol": 1e-7,
            "fatol": 1e-7,
        }
        self.actor_optimizer = optimizers.SciPyOptimizer(
            opt_method="SLSQP", opt_options=opt_options
        )
        self.critic_optimizer = optimizers.SciPyOptimizer(
            opt_method="SLSQP", opt_options=opt_options,
        )
    
    def initialize_actor_critic(self):
        self.critic = CriticTrivialSeminar(self.running_objective, self.sampling_time)

        self.actor = ActorSeminar(
            self.prediction_horizon,
            self.dim_input,
            self.dim_output,
            self.control_mode,
            self.action_bounds,
            action_init=self.action_init,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            critic=self.critic,
            running_objective=self.running_objective,
            model=self.actor_model,
        )


    def initialize_visualizer(self):

        self.animator = animators.Animator3WRobotNI(
            objects=(
                self.simulator,
                self.system,
                self.nominal_controller,
                self.controller,
                self.datafiles,
                self.logger,
                self.actor_optimizer,
                self.critic_optimizer,
                self.running_objective,
                self.scenario,
            ),
            pars=(
                self.state_init,
                self.action_init,
                self.time_start,
                self.time_final,
                self.state_init,
                self.xMin,
                self.xMax,
                self.yMin,
                self.yMax,
                self.control_mode,
                self.action_manual,
                self.v_min,
                self.omega_min,
                self.v_max,
                self.omega_max,
                self.Nruns,
                self.no_print,
                self.is_log,
                0,
                [],
            ),
        )
        
    def execute_pipeline(self, **kwargs):
        self.load_config()
        self.setup_env()
        self.__dict__.update(kwargs)
        self.initialize_system()
        self.initialize_predictor()
        self.initialize_safe_controller()
        self.initialize_models()
        self.initialize_objectives()
        self.initialize_optimizers()
        self.initialize_actor_critic()
        self.initialize_controller()
        self.initialize_simulator()
        self.initialize_logger()
        self.initialize_scenario()
        if not self.no_visual and not self.save_trajectory:
            self.initialize_visualizer()
            self.main_loop_visual()
        else:
            self.scenario.run()


In [ ]:
Pipeline3WRobotNISeminar().execute_pipeline()

In [ ]:
class Pipeline3WRobotNISeminarCasadi(Pipeline3WRobotNI):
    def initialize_optimizers(self):

        opt_options = {
            "print_time": 0,
            "ipopt.max_iter": 200,
            "ipopt.print_level": 0,
            "ipopt.acceptable_tol": 1e-7,
            "ipopt.acceptable_obj_change_tol": 1e-2,
        }

        self.actor_optimizer = optimizers.CasADiOptimizer(
            opt_method="ipopt", opt_options=opt_options
        )
        self.critic_optimizer = optimizers.CasADiOptimizer(
            opt_method="ipopt", opt_options=opt_options,
        )
        
Pipeline3WRobotNISeminarCasadi().execute_pipeline()